In [5]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from scipy.io import wavfile
import os
import pywt
from matplotlib import pyplot as plt
from scipy.signal import find_peaks

ModuleNotFoundError: No module named 'plotly'

In [ ]:
def load_and_process_ecg(
        folder_path: str, 
        file_index: int = 0,
    ):
    """
        Load and process ECG data from WAV file

        Parameters
        ----------
        folder_path : str
            Path to the folder containing WAV files
        file_index : int
        Default value is 0
    """
    wav_files = [f for f in os.listdir(folder_path) if f.endswith('.wav')]
    if not wav_files:
        raise ValueError("No WAV files found in specified folder")
    
    file_path = os.path.join(folder_path, wav_files[file_index])
    sample_rate, audio_data = wavfile.read(file_path)
    
    if audio_data.ndim > 1:
        audio_data = audio_data[:, 0]
    
    audio_data = audio_data / np.max(np.abs(audio_data))
    times = np.arange(len(audio_data)) / sample_rate
    
    return sample_rate, audio_data, times

In [ ]:
def apply_cwt(
        signal, 
        scales, 
        wavelet='mexh'
    ):
    """
        Apply Continuous Wavelet Transform

        Parameters
        ----------
        signal : array_like
            Input signal
        scales : array_like
        Default value is np.arange(1, 200)
        wavelet : str
            Default value is'mexh'
    """
    coef, freqs = pywt.cwt(signal, scales, wavelet)
    return coef

In [ ]:
def plot_cwt_analysis(
        times, 
        signal, 
        cwt_coef, 
        scales
    ) -> go.Figure:
    """
        Create plots for original signal and CWT scalogram
        
        Parameters
        ----------
        times : array_like
            Input signal times
        signal : array_like
            Input signal
        cwt_coef : array_like
        scales : array_like
            Default value is np.arange(1, 200)

    """
    # Create figure with secondary y-axis
    fig = go.Figure()

    # Add original ECG signal
    fig.add_trace(
        go.Scatter(
            x=times,
            y=signal,
            name='ECG Signal',
            line=dict(color='blue')
        )
    )

    # Add CWT scalogram as a heatmap
    fig.add_trace(
        go.Heatmap(
            z=cwt_coef,
            x=times,
            y=scales,
            colorscale='Jet',
            name='CWT',
            yaxis='y2'
        )
    )

    # Update layout
    fig.update_layout(
        title='ECG Signal Analysis with CWT',
        xaxis_title='Time (s)',
        yaxis_title='Amplitude',
        yaxis2=dict(
            title='Scale',
            overlaying='y',
            side='right'
        ),
        height=800  # Make the plot taller for better visibility
    )

    return fig

In [ ]:
def analyze_cwt_peaks(
        cwt_coef, 
        times, 
        scales, 
        threshold=0.8
    ):
    """
        Analyze peaks in CWT coefficients

        Parameters
        ----------
        cwt_coef : array_like
        times : array_like

    """
    # Normalize CWT coefficients
    cwt_norm = np.abs(cwt_coef) / np.max(np.abs(cwt_coef))
    
    # Find peaks in CWT coefficients at different scales
    peaks_dict = {}
    for i, scale in enumerate(scales):
        peaks, _ = find_peaks(cwt_norm[i, :], height=threshold, distance=len(times)//20)
        peaks_dict[scale] = times[peaks]
    
    return peaks_dict


In [ ]:

# Main execution
folder_path = "./ECG"

# Load and process ECG data
sample_rate, audio_data, times = load_and_process_ecg(folder_path)

# Define scales for CWT
num_scales = 64
scales = np.arange(1, num_scales)

# Apply CWT
cwt_coef = apply_cwt(audio_data, scales)

# Create visualization
fig = plot_cwt_analysis(times, audio_data, cwt_coef, scales)
fig.show()


NameError: name 'load_and_process_ecg' is not defined

In [ ]:

# Analyze peaks in CWT coefficients
peaks_dict = analyze_cwt_peaks(cwt_coef, times, scales)

# Create a plot showing peaks at different scales
fig_peaks = go.Figure()

# Plot original signal
fig_peaks.add_trace(
    go.Scatter(
        x=times,
        y=audio_data,
        name='ECG Signal',
        line=dict(color='blue', width=1)
    )
)
fig_peaks.show()

In [ ]:
fig_peaks = go.Figure()

# Plot peaks at different scales
colors = px.colors.qualitative.Set3
for i, (scale, peak_times) in enumerate(peaks_dict.items()):
    if len(peak_times) > 0:  # Only plot if peaks were found at this scale
        fig_peaks.add_trace(
            go.Scatter(
                x=peak_times,
                y=[scale / 10] * len(peak_times),  # Scale down for visibility
                mode='markers',
                name=f'Scale {scale}',
                marker=dict(
                    size=8,
                    color=colors[i % len(colors)]
                )
            )
        )

fig_peaks.update_layout(
    title='ECG Signal with CWT Peaks at Different Scales',
    xaxis_title='Time (s)',
    yaxis_title='Amplitude / Scale',
    height=600
)
fig_peaks.show()


In [ ]:

# Additional analysis: Find dominant scales
dominant_scales = np.argmax(np.abs(cwt_coef), axis=0)
scale_frequencies = np.bincount(dominant_scales) / len(dominant_scales)

# Plot scale distribution
fig_scales = px.bar(
    x=np.arange(len(scale_frequencies)),
    y=scale_frequencies,
    title='Distribution of Dominant Scales',
    labels={'x': 'Scale', 'y': 'Frequency'}
)
fig_scales.show()
